In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import time

In [2]:
# Initialize connection to the MT5 terminal
def connect_to_mt5(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        print("Initialization failed")
        mt5.shutdown()
        exit()
    print(f"Connected to account #{mt5.account_info().login}")

# Fetch historical data
def get_historical_data(symbol, timeframe, n_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n_bars)
    if rates is None:
        print(f"Failed to get data for {symbol}. Error:", mt5.last_error())
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# Calculate MACD
def calculate_macd(df):
    macd = df.ta.macd(close='close', fast=12, slow=26, signal=9)
    df = pd.concat([df, macd], axis=1)
    return df

# MACD Strategy
def macd_strategy(df):
    if df.iloc[-2]['MACD_12_26_9'] < df.iloc[-2]['MACDs_12_26_9'] and \
       df.iloc[-1]['MACD_12_26_9'] > df.iloc[-1]['MACDs_12_26_9']:
        return "Buy"
    elif df.iloc[-2]['MACD_12_26_9'] > df.iloc[-2]['MACDs_12_26_9'] and \
         df.iloc[-1]['MACD_12_26_9'] < df.iloc[-1]['MACDs_12_26_9']:
        return "Sell"
    return "Hold"

def place_order(symbol, action, volume):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Could not retrieve tick data for {symbol}")
        return False

    order_type = mt5.ORDER_TYPE_BUY if action == "Buy" else mt5.ORDER_TYPE_SELL
    price = tick.ask if action == "Buy" else tick.bid

    # Ensure volume is within limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info:
        volume = max(symbol_info.volume_min, min(volume, symbol_info.volume_max))
        if (volume - symbol_info.volume_min) % symbol_info.volume_step != 0:
            volume = round(volume / symbol_info.volume_step) * symbol_info.volume_step

    order = {
        'action': mt5.TRADE_ACTION_DEAL,
        'symbol': symbol,
        'volume': volume,  # Adjusted volume
        'type': order_type,
        'price': price,
        'deviation': 20,  # Maximum slippage in points
        'magic': 123456,
        'comment': f"Python {action} Order",
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling': mt5.ORDER_FILLING_RETURN,  # Change to RETURN
    }
    
    result = mt5.order_send(order)
    
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"{action} order executed successfully: {result}")
    else:
        print(f"{action} order failed, retcode: {result.retcode}")
        print("Full result:", result)

# Main function
def main():
    login = 48690778
    password = "FX3UU.#XhsVNi_w"
    server = "HFMarketsGlobal-Demo"
    symbol = "EURUSD"
    timeframe = mt5.TIMEFRAME_M1
    n_bars = 50
    volume = 0.01

    # Connect to MT5
    connect_to_mt5(login, password, server)

    # Ensure the symbol is available
    if not mt5.symbol_select(symbol, True):
        print(f"Symbol {symbol} not found")
        mt5.shutdown()
        exit()

    last_signal = "Hold"

    # Fetch and analyze live data
    while True:
        df = get_historical_data(symbol, timeframe, n_bars)
        if df is not None:
            df = calculate_macd(df)
            signal = macd_strategy(df)
            print(f"Time: {df.iloc[-1]['time']}, Signal: {signal}")

            if signal != "Hold" and signal != last_signal:
                if place_order(symbol, signal, volume):
                    last_signal = signal
        
        time.sleep(60)  # Wait 1 minute for the next analysis

    mt5.shutdown()

In [ ]:
# Run the script
if __name__ == "__main__":
    main()

Connected to account #48690778
Time: 2025-01-24 23:58:00, Signal: Hold
